# VIF - Visual Information Fidelity

VIF is a well-adopted image quality metric based on the premise that quality is complementary to the measure of information fidelity loss. In its original form, the VIF score is measured as a loss of fidelity combining four scales. In VMAF, we adopt a modified version of VIF where the loss of fidelity in each scale is included as an elementary metric. See [1](https://ieeexplore.ieee.org/abstract/document/1576816).

 - [1] H. Sheikh and A. Bovik, “Image Information and Visual Quality,” IEEE Transactions on Image Processing, vol. 15, no. 2, pp. 430–444, Feb. 2006.

## Example using ffmpeg

libvmaf is integrated into FFmpeg, to create complex filters to calculate VIF. This is an example where the distorted video is upscaled to the resolution of the reference video (360p to 1080p). The 'bicubic' interpolation method is used, see more on [FFmpeg](https://www.ffmpeg.org/ffmpeg-scaler.html). 

In [ ]:
!mkdir -p results/vif  

In [ ]:
! mkdir -p results/vif && \
ffmpeg -r 25 -i videos/dist/vp9/bbb_640x360_CRF30.webm \
       -r 25 -i videos/ref/bbb_1920x1080.mp4 \
       -lavfi "[0:v]scale=1920:1080:flags=bicubic,setpts=PTS-STARTPTS[distorted]; \
            [1:v]setpts=PTS-STARTPTS[reference]; \
            [distorted][reference]libvmaf=log_fmt=json:log_path=results/vif/bbb_vif_scaled.json:model='path=/vmaf/model/vmaf_float_v0.6.1.json'" -f null -   

## Graphical representation.

In this section, we plot the VIF values obtained per frame of the result obtained in Example 1. The arithmetic mean of the obtained values is also shown. 

In [ ]:
import pandas as pd
import json
import plotly.express as px

path_file='results/vif/bbb_vif_scaled.json'

with open(path_file) as f:
  data = json.load(f)

msssim_frames=[]

for frame in data['frames']:
    msssim_frames.append(frame['metrics']['vif_scale3'])
        
df=pd.DataFrame(msssim_frames, columns=['VIF'])

fig = px.line(df,x=df.index, y='VIF',
              title='VIF by frame',labels={"index": "Frame"}, template='simple_white', markers=True)


fig.update_layout(
    title={
        'y':0.9,
        'x':0.5,
        'xanchor': 'center'},
    showlegend=True)

fig.show()